In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.tsa.stattools as stattools
import numpy.random as rnd   # to try: rnd.choice
import re

In [18]:
def find_number_after_substring(string, substring):
    match = re.search(f"{substring}(\d+)", string)
    if match:
        return int(match.group(1))
    else:
        return 0


def get_model(process_type):
    if   'u' in process_type: z_type = 'uniform'
    elif 'n' in process_type: z_type = 'normal'
    else:
        raise ValueError("z_type not properly chosen. Supported: u for uniform, n for normal")
        
    if   'b' in process_type: step_type = 'binary'
    elif 'c' in process_type: step_type = 'continuous'
    else:
        raise ValueError("step_type not properly chosen. Supported: b for binary, c for continuous")
        
    if   'a' in process_type: cummulation = 'additive'
    elif 'm' in process_type: cummulation = 'multiplicative'
    else:
        raise ValueError("cummulation not properly chosen. Supported: a for additive, m for multiplicative")
        
    AR = find_number_after_substring(process_type, 'AR')
    MA = find_number_after_substring(process_type, 'MA')

    return z_type, step_type, cummulation, AR, MA


def get_zs(z_type, Ns, Nr):
    if ((z_type == 0) | (z_type == 'u') | (z_type == 'uniform')):
        return np.matrix(rnd.uniform(0, 1, Ns*Nr)).reshape([Ns,Nr])
    
    elif ((z_type == 1) | (z_type == 'n') | (z_type == 'normal')):
        return np.matrix(rnd.normal(0, 1, Ns*Nr)).reshape([Ns,Nr])
    
    else:
        raise ValueError("Random variable type not properly chosen.")
    
    
def get_steps(step_type, zs, mu = 0, dt = 1/252, sigma = 1, p = 0.5, lam = [], phi = []):
    Nt = zs.shape[0]
    Nr = zs.shape[1]
    AR = len(lam)
    MA = len(phi)
    
    if ((step_type == 0) | (step_type == 'b') | (step_type == 'binary')):
        return mu * dt + np.sign(p - zs) * sigma * np.sqrt(dt)
    
    elif ((step_type == 1) | (step_type == 'c') | (step_type == 'continuous')):
        ret = mu * dt  +  zs * sigma * np.sqrt(dt)
        
        if MA > 0:
            retro = zs
            for rank in range(1, MA+1):
                retro = np.concatenate((np.zeros([1, Nr]), retro[0:-1, :]))
                ret += phi[rank-1] * retro * np.sqrt(dt)

        if AR > 0:
            ret += mu * dt * np.sum(lam)

            retro = mu * dt * np.ones([AR, Nr])
            for t in range(Nt):
                ret[t, :] -= np.dot(lam, retro)             
                retro = np.concatenate((ret[t, :], retro[0:-1, :]))


        return ret

    else:
        raise ValueError("Step type not properly chosen.")
        
        
def get_prices(steps, cummulation, S0 = 1):
    [Ns, Nr] = steps.shape
    additive = np.zeros([Ns+1, Nr])
    
    for k in range(Ns):
        additive[k+1, :] = additive[k, :] + steps[k, :]
    
    if ((cummulation == 0) | (cummulation == 'a') | (cummulation == 'additive')):
        return additive + S0
    
    elif ((cummulation == 1) | (cummulation == 'm') | (cummulation == 'multiplicative')):
        return np.exp(additive) * S0
        
    else:
        raise ValueError("Cummulation type not properly chosen.")
        

In [19]:
def MC(process_type, Ns, Nr, S0 = 1, mu = 0, dt = 1/252, sigma = 1, draw = 5, p = 0.5, lam = [], phi = [], returned = 'psz'):
    z_type, step_type, cummulation, AR, MA = get_model(process_type)
    if AR != len(lam): raise ValueError("lam length must be the same as AR rank")
    if MA != len(phi): raise ValueError("phi length must be the same as MA rank")
    
    zs = get_zs(z_type, Ns, Nr)
    steps = get_steps(step_type, zs, mu = mu, dt = dt, sigma = sigma, p = p, lam = lam, phi = phi)
    prices = get_prices(steps, cummulation, S0 = S0)
    
    if draw > 0:
        if draw > Nr: draw = Nr
        if cummulation == 'multiplicative':
            plt.yscale("log")
        for run in range(draw):
            plt.plot(prices[:,run])
        plt.show()
        
    ret = pd.DataFrame({'RunNum':range(Nr)})
    ret = ret.join(pd.DataFrame({'StepNum':range(Ns+1)}), how = 'cross')
    
    if 'p' in returned:
        ret['Price'] = prices[ret['StepNum'], ret['RunNum']]
    if 's' in returned:
        ret.loc[ret['StepNum'] > 0, 'Step'] = steps[ret.loc[ret['StepNum'] > 0, 'StepNum'] - 1,
                                                 ret.loc[ret['StepNum'] > 0, 'RunNum']].transpose()
    if 'z' in returned:
        ret.loc[ret['StepNum'] > 0, 'Z'] = zs[ret.loc[ret['StepNum'] > 0, 'StepNum'] - 1,
                                           ret.loc[ret['StepNum'] > 0, 'RunNum']].transpose()
    return ret

In [1]:
def acf_single(data, order):
    d1 = data[:-order][:] if (order > 0) else data
    d2 = data[order:][:].reset_index(drop=True) if (order > 0) else data
    
    sig1 = np.sqrt(np.sum(np.square(d1 - np.mean(d1, axis = 0)), axis = 0))
    sig2 = np.sqrt(np.sum(np.square(d2 - np.mean(d2, axis = 0)), axis = 0))
    
    return np.sum((d1 - np.mean(d1, axis = 0)) * (d2 - np.mean(d2, axis = 0)), axis = 0) / sig1 / sig2


def acf(data, max_order = 15):
    if len(data.shape) > 1: runs = data.shape[1]
    else: runs = 1
    res = np.zeros([max_order, runs])
    for i in range(max_order):
        res[i, :] = acf_single(data, i)
    return res


def draw_acf(data, max_order = 15, stat_tools = False):
    if len(data.shape) > 1: 
        runs = data.shape[1]
    else:
        runs = 1
        
    if stat_tools:
        first_f = stattools.pacf(data.loc[:, 'TS' + str(1)])
        f = np.zeros([len(first_f), runs])
        for run in range(1,runs+1):
            f[:, run-1], confint = stattools.pacf(data.loc[:, 'TS' + str(run)], alpha = 0.05)
            print(confint)
            plt.figure()
            plt.bar(range(len(first_f)), f[:, run - 1])
    else:
        f = acf(data, max_order)
        for run in range(runs):
            plt.figure()
            plt.bar(range(max_order), f[:,run])